Recursive GoIndex Downloader by atlonxp

This code was created and improved by adapting the code from pankaj260 https://colab.research.google.com/drive/1tmsLGuswIZIZ_oM35EMW8TbJ6pQPt1rY#scrollTo=3bCnUMUg_SoT&forceEdit=true&sandboxMode=true

Features:
*   Recursive crawler (atlonxp)
*   Download all folders and files in a given url (atlonxp)
*   Download all folders and files in in sub-folders (atlonxp)
*   Adaptive delay in fetching url (atlonxp)
*   Store folders/files directly to your Google Drive (pankaj260)
*   Folders and files exclusion filters



In [1]:
# Mounting Google Drive, ignore this section if you don't want to 
# save on your Google Drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Install dependencies

!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [37.4 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [839 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,811 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1,183 kB]
Get:13 http

In [0]:
# Import dependencies

import os

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait

In [0]:
# Functions

def get_page_source(url, browser, delay, level, verbose=False):
    try:
        if verbose:
            print('\t' * level, url)
        browser.get(url)
        list_elm = WebDriverWait(browser, delay).until(lambda x: x.find_element_by_xpath('//*[@id="list"]/li[1]'))
        return browser.page_source
    except TimeoutException:
        raise Exception('Unable to load this website')


def check_exclusion(url, exclusions):
    for exc in exclusions:
        if exc in url:
            return True
    return False


def crawler(url, downloading_dict, level, exclusions, max_delay=60, verbose=False):
    page_source = get_page_source(url, browser, max_delay, level, verbose)
    soup = BeautifulSoup(page_source, 'html.parser')

    for fileFolders in soup.select('li.mdui-list-item a'):
        if (fileFolders.contents[1].text.find("folder_open") > -1):
            tempFolderName = fileFolders.contents[1].text
            folderName = tempFolderName.replace("folder_open", "").strip().strip().replace("/", "")
            folder_url = "https://edu.tuts.workers.dev" + fileFolders['href']

            if not check_exclusion(folder_url, exclusions):
                downloading_dict[folderName] = {
                    'type': 'folder',
                    'url': folder_url,
                    'child': crawler(folder_url, {}, level + 1, exclusions)
                }
        else:
            tempFileName = fileFolders.contents[1].text
            fileName = tempFileName.replace("insert_drive_file", "").strip().replace("/", "")
            file_url = "https://edu.tuts.workers.dev" + fileFolders['href'].replace("?a=view", "")

            if not check_exclusion(file_url, exclusions):
                downloading_dict[fileName] = {
                    'type': 'file',
                    'url': file_url
                }

    return downloading_dict


def downloader(downloading_dict, level, path, verbose=False):
    for key, value in downloading_dict.items():
        if value['type'] == 'folder':
            path_abs_dir = os.path.join(path, key)
            if verbose:
                print('\t' * level, path_abs_dir)

            try:
                if not os.path.exists(path_abs_dir):
                    os.mkdir(path_abs_dir)
            except Exception:
                pass
            downloader(value['child'], level + 1, path_abs_dir, verbose)
        else:
            file_abs_path = os.path.join(path, key)
            if verbose:
                print('\t' * level, file_abs_path)
            if os.path.exists(file_abs_path):
                print("skipping => " + os.path.join(path, key))
            else:
                r = requests.get(value['url'], stream=True)
                with open(file_abs_path, "ab+") as f:
                    f.write(r.content)

In [0]:
# Browser Initialization

chromedriver = "/usr/bin/chromedriver"
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(chromedriver, options=chrome_options)

In [0]:
exclusions = ['__MACOSX/']
max_waiting_delay = 60

destination = "/content/drive/My Drive/Knowledge/_Trainings/_download"
download_url = 'https://edu.tuts.workers.dev/Udacity%20-%20Collections%20%5B300%20GB%5D/Udacity%20-%20Developing%20Scalable%20App%20in%20Python/'

downloading_dict = crawler(download_url, {}, 0, exclusions, verbose=True)
downloader(downloading_dict, 0, path=destination, verbose=True)
print('All done, Voila!')

 https://edu.tuts.workers.dev/Udacity%20-%20Collections%20%5B300%20GB%5D/Udacity%20-%20Developing%20Scalable%20App%20in%20Python/
 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/01 - Welcome to the Course.mp4
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/01 - Welcome to the Course.srt
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/02 - Interview With Urs Pt 1.mp4
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/02 - Interview With Urs Pt 1.srt
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/03 - History of Scaling.mp4
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalability Basics/03 - History of Scaling.srt
	 /content/drive/My Drive/Knowledge/_Trainings/_download/Lesson 1 - Scalabilit